<p style="font-size:32px; font-weight: bolder; text-align: center"> Atomistic machine learning:<br/>an introduction </p>
<p style="text-align: center"><i> authored by: <a href="mailto:michele.ceriotti@gmail.com"> Michele Ceriotti </a></i></p>

This course provides a hands-on introduction to the machine-learning techniques, and to
their application to chemical and materials modeling. The course covers both the 
fundamentals and some more advanced techniques, balancing general concepts and 
techniques that are specific to atomistic modeling. 

The course is organized as a series of Jupyter notebooks, each covering one module:

1. [Linear methods (PCA, ridge regression)](./01-Linear_methods.ipynb)
2. [Structural and chemical representations](./02-Representations.ipynb)
3. [Equivariant features and regression](./03-Equivariant_regression.ipynb)

### Packages and dependencies

In [ ]:
import numpy as np
import scipy as sp

import matplotlib as mpl
import matplotlib.pyplot as plt

from ipywidgets import FloatSlider, IntSlider, Checkbox, Dropdown, HBox, Layout, HTML

from scwidgets.check import (
    Check,
    CheckRegistry,
    assert_numpy_allclose,
    assert_numpy_floating_sub_dtype,
    assert_shape,
    assert_type,
)

import ase
import chemiscope

from scwidgets.code import ParametersPanel, CodeInput
from scwidgets.cue import CueObject, CueFigure
from scwidgets.exercise import CodeExercise, TextExercise, ExerciseRegistry

# Course how-to

The course contains a combination of text-book-style explanations, simple mathematical derivations, and interactive widgets in which you need to manipulate functions or atomic structures, and/or enter short snippets of code. The course material is conceived so that minimal amounts of prior knowledge about Python or Jupyter notebooks is needed, but you will have to do _some_ coding. 
 If your Python is rusty, you should look for some online course or tutorials to get up to speed. You don't need to know much about object oriented programming, but you should know enough to initialize a class and use class methods. Looking up examples and documentation online is fine, and even encouraged.

 At the top of each notebook you will find a box to enter your name. This will also be used to create a file in which you can save (and load from) the answers you have given to exercises and text widgets. We suggest to use `SurnameName`, which will lead to a file named `module_XX-SurnameName.json`. If you are looking at these notebooks as part of a formal course, you will be able to send these for grading. 

> **_NOTE:_** The state of the widgets is not saved automatically, so remember to click "Save" once you are done with each exercise._

In [ ]:
exercise_registry = ExerciseRegistry(filename_prefix="module_00")
exercise_registry

In [ ]:
check_registry = CheckRegistry()  # this is needed to coordinate code checking

Throughout the notebook you will find highlighted paragraphs with a bold number.  These are instructions that you need to follow to answer some exercises, or comment on what you observe in interactive widgets. 

This might imply typing some open answer commenting the results you are seeing, or reporting the result of some simple derivation. Please keep this as tidy and clean: be concise and to the point, this is to help you collect your thoughts as much as to provide evaluation.  

In [ ]:
answerbox = TextExercise(
    description="""
    For example, this is a box to answer an open question: experiment with 
    saving and loading the answer: should be pretty intuitive but it's 
    better to figure it out here. """,
    key="00",
    title="Text Box Exercise",
    exercise_registry=exercise_registry,
)
display(answerbox)

# Appmode, hide inputs, and dependencies

The notebooks work on both Jupyter classic notebooks and in JupyterLab. To help you focus on the content and not on the quirks of building an interactive notebook, they are designed to function without the need for you to enter code into input cells, but only inside dedicated widgets. To avoid being distracted, you can hide the input cells using the *Appmode* Jupyter plugin (for Jupyter classic), which you can activate by clicking on the corresponding button
<img src="figures/appmode_button.png" width="80"/>, or the *hide code* extension for JupyterLab, which you can activate clicking the "hide input" icon <img src="figures/hidecode_button.png" height="20"/>. After clicking on the "hide input" icon, please restart and rerun the notebook by clicking on the <img src="figures/rerunnotebook_button.png" height="20"/> icon. Students with stronger programming background, who want to look "under the hood", can look at the source code at any time switching on and off these extensions

> **_NOTE:_** Save your progress before doing so, as some content might get lost!

If you are using these notebooks for a course using your school's jupyter server, you should find yourself in a fully-configured environment, or you should ask the instructor to have it set up for you. If instead you are using them on your own system, you may need to install several prerequisites. In this case, from the main folder, run `pip install -r requirements.txt`, or a similar command with your favourite Python package manager.

# Interactive widgets

The notebooks often contain interactive widgets that can be manipulated by changing some slider values, to visualize the concepts being discussed. Usually these don't require any coding, just to follow some instructions and/or to experiment with the values do develop a more intuitive understanding of the significance of an equation, or to test its limits.

In [ ]:
def plot_sine(ax, w, a, abval):
    xgrid = np.linspace(0,5,100)
    if abval:
        ax.plot(xgrid, np.abs(a*np.sin(xgrid*w)))
        ax.set_ylabel(r"$|\sin x|$")
    else:
        ax.plot(xgrid, a*np.sin(xgrid*w))
        ax.set_ylabel(r"$\sin x$")
    ax.set_xlabel("$x$")

sine_parameterbox = ParametersPanel(
    w = FloatSlider(value=2.0,min=0,max=10,step=0.1,description=r'$\omega$'),
    a = FloatSlider(value=2.0,min=1,max=100,step=2,description=r'$A$'),
    abval=Checkbox(False, description=r'Absolute value')
)   

sine_fig, sine_ax = plt.subplots(1,1,figsize=(5,3.5))

def sine_update(cue_exercise):
    w,a,abval = cue_exercise.parameters.values()
    axes = cue_exercise.figure.get_axes()[0]  
    
    plot_sine(axes,w,a,abval)

sine_demo = CodeExercise(
    parameters=sine_parameterbox,
    outputs=sine_fig,
    update=sine_update,
    update_mode="continuous"
) 


The widget displays a sine function $y = A \sin \omega x$, that oscillates with a period $2\pi/\omega$ amd an amplitude spanning the range $[-A,A]$.
Optionally, you can compute (and plot) the absolute value of the function. 
When you modify the values of the parameters, the plot is automatically updated. 

In [ ]:
sine_demo.run_update()
display(sine_demo)

In [ ]:
# all the plotting can be contained within the update function
def sine_update(ex):
    w,a,abval = ex.parameters.values()
    ax = ex.figure.get_axes()[0]  

    xgrid = np.linspace(0,5,100)
    if abval:
        ax.plot(xgrid, np.abs(a*np.sin(xgrid*w)))
        ax.set_ylabel(r"$|\sin x|$")
    else:
        ax.plot(xgrid, a*np.sin(xgrid*w))
        ax.set_ylabel(r"$\sin x$")
    ax.set_xlabel("$x$")
    
sine_parameterbox_click = ParametersPanel(
    w = FloatSlider(value=2.0,min=0,max=10,step=0.1,description=r'$\omega$'),
    a = FloatSlider(value=2.0,min=1,max=100,step=2,description=r'$A$'),
    abval=Checkbox(False, description=r'Absolute value')
)
    
sine_output_click =  CueFigure(plt.subplots(1,1,figsize=(5,3.5))[0])

sine_demo_click = CodeExercise(
            parameters=sine_parameterbox_click,
            outputs=sine_output_click,
            update=sine_update,
            update_mode="manual"       
) 

Some more time-consuming visualizations cannot be updated on-the-fly. In these cases, you'll find an _Update_ button that you can press after you have set all the parameters to your liking. 

In [ ]:
display(sine_demo_click)

In [ ]:
structures = [ ase.Atoms("OH2", positions=np.asarray([[0,0,0], [0.7,0.5,0],  [-0.7,0.5,0]])*np.asarray([[0,s1,s2]]).T) for s1 in [0.9,1,1.1,1.2] for s2 in [0.9,1,1.1,1.2] ]
properties = np.asarray([(s1, s2, (s1-1)**2+(s2-1)**2) for s1 in [0.9,1,1.1,1.2] for s2 in [0.9,1,1.1,1.2] ] )
properties = { "stretch_1":  properties[:,0]-1, "stretch_2":  properties[:,1]-1, "energy" : properties[:,2] }
cs = chemiscope.show(structures, properties, meta= {"name": "Water molecules", "description": "Stretched water molecules with completely made up energy"},mode="structure")

We can also display atomic structures in a dedicated [chemiscope](https://chemiscope.org/) widget. Experiment with the settings for visualizing the structure by clicking on the <img src="figures/chemiscopesettings_button.png" height="20"/> icon. Also try clicking on the structure information <img src="figures/chemiscopeinfo_button.png" height="20"/> field and the play button below the structure panel.

In [ ]:
display(cs)

Chemiscope also allows to associate a list of properties with the corresponding structures, displaying an interactive map that allows, by clicking, to view the corresponding structure. You can change the visualization settings if you want, and save a snapshot of either the plot or the structure.  

# Code widgets

Some exercises require inputting short code snippets into a dedicated code widget. This code forms the body of a function, whose return value can then be checked by plotting, or by comparison with known reference values. The function is run as a stand-alone Python code, so you can only use variables and modules that are defined or imported within the code widget. Each code widget has its separate scope.

In [ ]:
def exercise_function(x, a, w, f_abs):
    """
    Computes the sine function with a given frequency and amplitude, optionally taking the absolute value

    :param x: the input variable
    :param a: the amplitude
    :param w: the frequency
    :param f_abs: bool: take the absolute value?
    
    :return: abs(a*sin(w x)) if f_abs else a*sin(w x)
    """
    # Write your solution, then click on the button below to update the plotter and check against the reference values
    from math import sin
    if f_abs:
        return abs(a*sin(w*x))
    else:
        return a*sin(w*x)

code_fig = plt.figure()
code_fig.add_subplot(111)

def code_update(code_example):
    w,a,f_abs = code_example.parameters.values()
    
    ax = code_example.figure.get_axes()[0]
    func = code_example.code
    xgrid = np.linspace(-10,10,500)    
    ygrid = np.asarray([ func(x, a, w, f_abs) for x in xgrid])
    ax.plot(xgrid, ygrid, 'b-')
    ax.set_xlabel(r"x") 
    ax.set_ylabel("y")
    
code_parameterbox = ParametersPanel(
    w=FloatSlider(value=2.0,min=0,max=10,step=0.1,description=r'$\omega$'),
    a=FloatSlider(value=2.0,min=1,max=100,step=2,description=r'$A$'),
    abval=Checkbox(False, description=r'Absolute value')
)

code_demo = CodeExercise(
    code=exercise_function,
    check_registry=check_registry,
    outputs=code_fig,
    parameters=code_parameterbox,
    update=code_update,
    update_mode="manual",
    description="""
    Nothing to do here - code is already functional - but you can play around to change
    it and see what happens!""",    
    key="01",
    title="Code Box Exercise",
    exercise_registry=exercise_registry,
)

check_registry.add_check(
    code_demo,
    inputs_parameters=[
        {"x" : 2, "w":1, "a":2, "f_abs":False}, 
        {"x" : 1, "w":-0.5, "a":0.2, "f_abs":True},
                   ],
    outputs_references=[(1.8185948536513634,), (0.0958851077208406,)],
    asserts=[
        assert_numpy_allclose,
    ],
)

Many of the answers you will have to give involve simple coding exercises. 
To answer these questions you just need to write a function that performs the task specified. In this case, the function is already written and computes the sine. 
Often you'll see these exercises associated with a visualizer that displays e.g. the function you compute, helping you understand if there is an error, and just familiarize yourself with the concepts. 

To help you check whether the code is producing correct results, you will usually also find a _Check_ button, that runs a few tests and returns either a confirmation that the core has run correctly, or some errors that may help you correct mistakes.

Finally, remember to _Save_ your code before moving on to the next exercise. 

In [ ]:
display(code_demo)